In [5]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point,LineString
df = pd.read_csv('data/sharks_cleaned.csv')

import os

os.environ['PROJ_LIB'] = r"C:\Users\eleanor.sluyter-mead\AppData\Local\Continuum\anaconda3\Library\share"

In [6]:
def make_geometry(pts):
    if len(pts) == 1:
        return pts[0]
    else:
        return LineString(pts)

df['point'] = df.apply(lambda r: Point(r.longitude,r.latitude), axis=1)
sorteddf = df.sort_values(by=['name', 'datetime'])
sharkdf = sorteddf.groupby(['name','species'])['point'].apply(list).reset_index()
sharkdf = sharkdf[sharkdf.apply(lambda r: len(r.point) > 1, axis=1)]
sharkdf['geometry'] = sharkdf.apply(lambda r: make_geometry(r.point),axis=1)

In [7]:
sharkdf.head()

,name,species,point,geometry
0,AB,Tiger Shark (Galeocerdo cuvier),"[POINT (-80.37539 30.49353), POINT (-80.36515 ...","LINESTRING (-80.37539 30.49353, -80.36515 30.4..."
1,ANZAC,Tiger Shark (Galeocerdo cuvier),"[POINT (113.48683 -22.84421), POINT (113.5135 ...","LINESTRING (113.48683 -22.84421, 113.5135 -22...."
2,Adelaide,Tiger Shark (Galeocerdo cuvier),"[POINT (113.71675 -23.10293), POINT (113.63252...","LINESTRING (113.71675 -23.10293, 113.632523324..."
3,Al,Tiger Shark (Galeocerdo cuvier),"[POINT (-80.23236999999999 31.92491), POINT (-...","LINESTRING (-80.23236999999999 31.92491, -80.1..."
4,Albert,White Shark (Carcharodon carcharias),"[POINT (20.53316999999998 -34.48659), POINT (2...","LINESTRING (20.53316999999998 -34.48659, 20.54..."


In [8]:
shark_gdf = gpd.GeoDataFrame(sharkdf[['name', 'geometry','species']])
shark_gdf.crs = {'init': 'epsg:4326'}

In [9]:
shap = folium.Map(tiles='cartodbpositron')
#folium.GeoJson(shark_gdf).add_to(shap)
#shap.save('maps/shaptest.html')

In [10]:
#list of species
#for loop to go through list of species and create species-specific geojson

species = list(df['species'].unique())
species[:5]

['White Shark (Carcharodon carcharias)',
 'Tiger Shark  (Galeocerdo cuvier)',
 'Bull Shark (Carcharhinus leucas)',
 'Mako Shark (Isurus oxyrinchus)',
 'Blue Shark (Prionace glauca)']

In [11]:
for specie in species:
    speciedf = shark_gdf[shark_gdf.species == specie]
    gjspecies = folium.GeoJson(speciedf)
    fg = folium.FeatureGroup(name=specie)
    gjspecies.add_to(fg)
    fg.add_to(shap)
    
#making a for loop
#to cycle through each species
#and create a new df for each species
#based off the sharkdf

RuntimeError: b'no arguments in initialization list'

In [12]:
folium.LayerControl().add_to(shap)
shap.save('maps/shaptest.html')